#### importing the relevant packages and loading the data

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
# from sklearn.feature_selection import RFE

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

from scipy import stats

from xgboost import XGBRegressor

In [6]:
X = pd.read_csv("dengue_features_train.csv")
y = pd.read_csv("dengue_labels_train.csv")
test = pd.read_csv("dengue_features_test.csv")

#### Pre Processing

In [3]:
def impute(X):
    
    # remove the colum that has ~20% null values, also ranks low on feature importance
    X.drop(['ndvi_ne'], axis=1, inplace=True)
    
    # Filling the rest using linear interpolation
    X.interpolate(inplace=True)

def remove_outliers(df):
    return df[(np.abs(stats.zscore(df)) < 5).all(axis=1)]

def mape(Y_test, Y_pred, epsilon = 1):
    return np.mean(np.abs((Y_test - Y_pred + epsilon) / (Y_test + epsilon))) * 100

def extract_month(s):
    return int(s[5:7])

def city_indices(X):
    # city boolean encoding
    return X.city == 'sj'

def pre_process(X, trees = False):
    """
    Extracts the month out of date and converts it to a one hot
    Standardizes the numerical features
    """
    
    #Extracting month from the date
    months = X.week_start_date.apply(extract_month)

    # Removing the columns not required for classification
    X.drop(['city', 'year', 'weekofyear', 'week_start_date'], axis=1, inplace=True)

    # Standardizing the data
    if not trees:
        scaler = StandardScaler()
        X[X.columns] = scaler.fit_transform(X)

    # Month one hot features
    month_features = pd.get_dummies(months)
    X = X.join(month_features)

    # Alternatively use months as a discrete feature
    #X = X.join(months)
    
    return X

def seperate_cities_data(X, is_sj):

    # Seperating the cities data
    X_sj = X.loc[is_sj]
    X_iq = X.loc[~is_sj]
    
    return X_sj, X_iq

def get_y_labels(X_sj, X_iq, y):    
    
    y = y.total_cases    
    y_sj = y.loc[X_sj.index]
    y_iq = y.loc[X_iq.index]
    
    return y_sj, y_iq

def split(X_sj, X_iq, y_sj, y_iq):

    # train and test split
    sj_split_data = train_test_split(X_sj, y_sj, shuffle = False)
    iq_split_data = train_test_split(X_iq, y_iq, shuffle = False)

    return sj_split_data, iq_split_data

def process(X, y = pd.Series(), train = True, trees = False):
    
    is_sj = city_indices(X)
    if not trees:
        impute(X)
    X = pre_process(X, trees)
    
    #X = remove_outliers(X)
    X_sj, X_iq = seperate_cities_data(X, is_sj)
    if y.empty:
        return X_sj, X_iq
    
    y_sj, y_iq = get_y_labels(X_sj, X_iq, y)
    if not train:
        return X_sj, X_iq, y_sj, y_iq
    
    return split(X_sj, X_iq, y_sj, y_iq)

#### Random Model

In [4]:
data = process(X,y)
(X_sj_train, X_sj_test, Y_sj_train, Y_sj_test), (X_iq_train, X_iq_test, Y_iq_train, Y_iq_test) = data

NameError: name 'StandardScaler' is not defined

In [ ]:
def random(Y_test, Y_train):
    y_p = np.full(len(Y_test), np.mean(Y_train))
    return mean_absolute_error(Y_test, y_p)

In [ ]:
random(Y_sj_test, Y_sj_train)

In [ ]:
random(Y_iq_test, Y_iq_train)

In [ ]:
Y_test = Y_sj_test.append(Y_iq_test)
Y_train = Y_sj_train.append(Y_iq_train)

In [ ]:
random(Y_test, Y_train)

#### Generalised Model

In [7]:
def general_model(clf, data):
    
    (X_sj_train, X_sj_test, Y_sj_train, Y_sj_test), (X_iq_train, X_iq_test, Y_iq_train, Y_iq_test) = data
    
    clf.fit(X_sj_train, Y_sj_train, eval_metric = mean_absolute_error)
    Y_sj_pred = clf.predict(X_sj_test)
    
    clf.fit(X_iq_train, Y_iq_train, eval_metric = mean_absolute_error)
    Y_iq_pred = clf.predict(X_iq_test)
    
    Y_pred = np.concatenate([Y_sj_pred, Y_iq_pred])
    Y_pred = Y_pred.astype(int).clip(0)
    Y_test = Y_sj_test.append(Y_iq_test)
    
    return mean_absolute_error(Y_test, Y_pred)

#### Baseline Model

In [8]:
data = process(X,y, trees=True)

In [9]:
model = XGBRegressor()
general_model(model, data)

18.26923076923077

#### tuned model

In [10]:
from sklearn.model_selection import cross_val_score

from sklearn.metrics import make_scorer

rf = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1) 

(X_sj_train, X_sj_test, Y_sj_train, Y_sj_test), (X_iq_train, X_iq_test, Y_iq_train, Y_iq_test) = data

cross_val_score(rf, X_sj_train, Y_sj_train, cv = 5, scoring=make_scorer(mean_absolute_error))

array([30.59434104, 49.11759507, 14.95902743, 32.26011295, 12.66011701])

In [11]:
cross_val_score(rf, X_iq_train, Y_iq_train, cv = 5, scoring=make_scorer(mean_absolute_error))

array([3.25106094, 7.87761068, 7.00398358, 5.8487702 , 3.53553   ])

#### time based cross validations

In [13]:
clf = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1)

In [23]:
cv_splits = TimeSeriesSplit(n_splits=3).split(X_sj_train, Y_sj_train)

cross_val_score(clf, X_sj_train, Y_sj_train, cv=cv_splits, scoring=make_scorer(mean_absolute_error))

array([40.78982837, 31.14011517, 12.58860924])

In [21]:
cv_splits = TimeSeriesSplit(n_splits=3).split(X_iq_train, Y_iq_train)
cross_val_score(clf, X_iq_train, Y_iq_train, cv=cv_splits, scoring=make_scorer(mean_absolute_error))

array([5.49918169, 8.7481604 , 4.26543863])

#### Tuned parameters

In [49]:
# Initialize XGB and GridSearch
# we find gamma=0 and max_depth=3, defaut parameters to be best, rest tuned parameters can be seen below
rf = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1) 

general_model(rf, data)

12.39010989010989

In [37]:
#'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
grid = {'colsample_bylevel' : [i/10.0 for i in range(1,11)]}

best_error = 100

i = 1

for g in ParameterGrid(grid):
    rf.set_params(**g)
    error = general_model(rf, data)
    # save if best
    if error < best_error:
        best_error = error
        best_grid = g

    if (i % 5 == 0):
        print (i)
    i += 1
    
print ("Best Error: %0.5f" % best_error)
print ("Grid:", best_grid)

5
10
Best Error: 12.31215
Grid: {'colsample_bylevel': 0.5}


#### Submission

In [25]:
X = pd.read_csv("dengue_features_train.csv")
y = pd.read_csv("dengue_labels_train.csv")
test = pd.read_csv("dengue_features_test.csv")
model = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1)

In [27]:
X_sj, X_iq, y_sj, y_iq = process(X, y, train = False, trees=True)
X_sj_test, X_iq_test = process(test, trees=True)

model.fit(X_sj, y_sj)
Y_pred_sj = model.predict(X_sj_test)

model.fit(X_iq, y_iq)
Y_pred_iq = model.predict(X_iq_test)

Y_pred = np.concatenate([Y_pred_sj, Y_pred_iq])

In [29]:
test = pd.read_csv("dengue_features_test.csv")
submission = test.loc[:,['city', 'year', 'weekofyear']].copy()
submission['total_cases'] = Y_pred.astype(int).clip(0)
submission.to_csv('submission_4.csv', index=False)